**CODE TO TRAIN QA-LSTM ON WORD EMBEDDINGS FOR BEST ANSWER PREDICTION. BASE CODE FROM** https://github.com/sujitpal/dl-models-for-qa/blob/master/src/qa-lstm.py

In [ ]:
from __future__ import division, print_function
from gensim.models import Word2Vec
from keras.callbacks import ModelCheckpoint

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

import numpy as np
import os

import pandas as pd

In [ ]:
WORD2VEC_EMBED_SIZE = 300
QA_EMBED_SIZE = 64

In [ ]:
'''DATA_DIR = os.path.join(project_folder, "qa-lstm"
MODEL_DIR = os.path.join(project_folder, "qa-lstm")
WORD2VEC_BIN = "GoogleNews-vectors-negative300.bin.gz"'''

SyntaxError: ignored

In [ ]:
QA_EMBED_SIZE = 64
BATCH_SIZE = 32
NBR_EPOCHS = 20

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
project_folder = "./drive/My Drive/csc2515-project/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd

questions_answers = pd.read_csv(os.path.join(project_folder, "Train_RemovedNAN_AddedNormScore.csv"))

In [ ]:
questions_answers['ShortenedA'] = questions_answers['Clean_Answer'].str.split()
questions_answers['ShortenedQ'] = questions_answers['Clean_Question'].str.split()

#SEQ_LEN_Q = 100
#SEQ_LEN_A = 100

questions_answers['ShortenedA'].head(10)
questions_answers['ShortenedQ'].head(10)

'''for idx, row in questions_answers.iterrows():
  questions_answers.at[idx, 'ShortenedA'] = row.Clean_Answer[0:SEQ_LEN_A]
  questions_answers.at[idx, 'ShortenedQ'] = row.Clean_Question[0:SEQ_LEN_Q]'''

questions_answers['ShortenedA'].head(10)
questions_answers['ShortenedQ'].head(10)

0    [this, is, something, ive, pseudosolved, many,...
1    [this, is, something, ive, pseudosolved, many,...
2    [this, is, something, ive, pseudosolved, many,...
3    [this, is, something, ive, pseudosolved, many,...
4    [this, is, something, ive, pseudosolved, many,...
5    [this, is, something, ive, pseudosolved, many,...
6    [this, is, something, ive, pseudosolved, many,...
7    [this, is, something, ive, pseudosolved, many,...
8    [this, is, something, ive, pseudosolved, many,...
9    [i, have, a, little, game, written, in, c, it,...
Name: ShortenedQ, dtype: object

In [ ]:
questions_answers['Labels'] = questions_answers.groupby(['QId'])['Score'].transform(max) == questions_answers['Score']
questions_answers['Best_Score'] = questions_answers[['Labels']] * 1

In [ ]:
q = questions_answers['ShortenedQ'].tolist()
a = questions_answers['ShortenedA'].tolist()
score = questions_answers['Best_Score'].tolist()
qa_pairs = []

for each_q, each_a, each_score in zip(q, a, score):
    #print (each_q, each_a)
    qa_pairs.append([each_q, each_a, each_score])

In [ ]:
question_maxlen = max([len(qa_pair[0]) for qa_pair in qa_pairs])
answer_maxlen = max([len(qa_pair[1]) for qa_pair in qa_pairs])
seq_maxlen = max([question_maxlen, answer_maxlen])

In [ ]:
print(question_maxlen)
print(answer_maxlen)
print(seq_maxlen)

1425
3707
3707


In [ ]:
import collections

def build_vocab(stories, qapairs, testqs):
    wordcounts = collections.Counter()
    for story in stories:
        for sword in story:
            wordcounts[sword] += 1
    for qapair in qapairs:
        #print(qapair[0])
        #split_q = qapair[0].split()
        #split_a = qapair[1].split()
        for qword in qapair[0]:
            wordcounts[qword] += 1
        for aword in qapair[1]:
            wordcounts[aword] += 1
    for testq in testqs:
        for qword in testq[0]:
            wordcounts[qword] += 1
        for aword in testq[1]:
            wordcounts[aword] += 1
    words = [wordcount[0] for wordcount in wordcounts.most_common()]
    word2idx = {w: i+1 for i, w in enumerate(words)}  # 0 = mask
    return word2idx

word2idx = build_vocab([], qa_pairs, [])
vocab_size = len(word2idx) + 1 # include mask character 0

In [ ]:
vocab_size

133776

In [ ]:
word2idx

{'the': 1,
 'to': 2,
 'a': 3,
 'i': 4,
 'is': 5,
 'and': 6,
 'of': 7,
 'in': 8,
 'it': 9,
 'that': 10,
 'this': 11,
 'you': 12,
 'for': 13,
 'have': 14,
 'be': 15,
 'with': 16,
 'but': 17,
 'if': 18,
 'on': 19,
 'can': 20,
 'not': 21,
 'as': 22,
 'are': 23,
 'or': 24,
 'an': 25,
 'my': 26,
 'do': 27,
 'use': 28,
 'so': 29,
 'code': 30,
 'like': 31,
 'there': 32,
 'what': 33,
 'from': 34,
 'your': 35,
 'using': 36,
 'will': 37,
 'would': 38,
 'how': 39,
 'when': 40,
 'which': 41,
 'all': 42,
 'im': 43,
 'one': 44,
 'any': 45,
 'some': 46,
 'want': 47,
 'its': 48,
 'by': 49,
 'just': 50,
 'way': 51,
 'at': 52,
 'then': 53,
 'get': 54,
 'need': 55,
 'am': 56,
 'should': 57,
 'file': 58,
 'was': 59,
 'only': 60,
 'more': 61,
 'has': 62,
 'me': 63,
 'c': 64,
 'other': 65,
 'also': 66,
 'dont': 67,
 'know': 68,
 'about': 69,
 'example': 70,
 'no': 71,
 'work': 72,
 'does': 73,
 'here': 74,
 'class': 75,
 'time': 76,
 'same': 77,
 'out': 78,
 'could': 79,
 'they': 80,
 'problem': 81,
 'we': 8

In [ ]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

def vectorize_qapairs(qapairs, word2idx, seq_maxlen):
    pos = 0
    Xq, Xa, Y = [], [], []
    for qapair in qapairs:
        if qapair[2]:
          pos += 1
        #split_q = qapair[0].split()
        #split_a = qapair[1].split()
        Xq.append([word2idx[qword] for qword in qapair[0]])
        Xa.append([word2idx[aword] for aword in qapair[1]])
        Y.append(np.array([1, 0]) if qapair[2] else np.array([0, 1]))
    return (pad_sequences(Xq, maxlen=seq_maxlen), 
            pad_sequences(Xa, maxlen=seq_maxlen),
            np.array(Y))

Xq, Xa, Y = vectorize_qapairs(qa_pairs, word2idx, seq_maxlen)

In [ ]:
# Save the vectorized pairs
import pickle

pickle_vectorized_qa_pairs_xq = os.path.join(project_folder, "qa-lstm/vectorized_qa_pairs_xq.pkl")
pickle_vectorized_qa_pairs_xa = os.path.join(project_folder, "qa-lstm/vectorized_qa_pairs_xa.pkl")
pickle_vectorized_qa_pairs_y = os.path.join(project_folder, "qa-lstm/vectorized_qa_pairs_y.pkl")

pickle.dump((Xq),open(pickle_vectorized_qa_pairs_xq,'wb'))
pickle.dump((Xa),open(pickle_vectorized_qa_pairs_xa,'wb'))
pickle.dump((Y),open(pickle_vectorized_qa_pairs_y,'wb'))

KeyboardInterrupt: ignored

In [ ]:
# Load the vectorized pairs
import pickle

pickle_vectorized_qa_pairs_xq = os.path.join(project_folder, "qa-lstm/vectorized_qa_pairs_xq.pkl")
pickle_vectorized_qa_pairs_xa = os.path.join(project_folder, "qa-lstm/vectorized_qa_pairs_xa.pkl")
pickle_vectorized_qa_pairs_y = os.path.join(project_folder, "qa-lstm/vectorized_qa_pairs_y.pkl")

Xq = pickle.load(open(pickle_vectorized_qa_pairs_xq, 'rb'))
Xa = pickle.load(open(pickle_vectorized_qa_pairs_xa, 'rb'))
Y = pickle.load(open(pickle_vectorized_qa_pairs_y, 'rb'))

In [ ]:
from sklearn.model_selection import train_test_split

# Find the split index, then iterate to next question since the the split should not be in the middle of a question
split_index = int(len(questions_answers) * 0.8)

split_Q = questions_answers['QId'].iloc[split_index]
while(split_index<len(questions_answers) and questions_answers['QId'].iloc[split_index] == split_Q):
  split_index += 1

Xqtrain = Xq[:split_index]
Xqtest = Xq[split_index:]

Xatrain = Xa[:split_index]
Xatest = Xa[split_index:]

Ytrain = Y[:split_index]
Ytest = Y[split_index:]

#Ytr = normalized_scores[:split_index]

In [ ]:
questions_answers_train = questions_answers[:split_index]
questions_answers_test = questions_answers[split_index:]

In [ ]:
print(Xqtrain.shape, Xqtest.shape, Xatrain.shape, Xatest.shape, 
      Ytrain.shape, Ytest.shape)

(86965, 3707) (21740, 3707) (86965, 3707) (21740, 3707) (86965, 2) (21740, 2)


In [ ]:
!pip install gensim

In [ ]:
# Get word embeddings file

EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-12-16 06:27:35--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.96.30
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.96.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  72.5MB/s    in 22s     

2020-12-16 06:27:58 (71.9 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
from gensim.models import Word2Vec, KeyedVectors
from gensim import models

print("Loading Word2Vec model and generating embedding matrix...")

word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
embedding_weights = np.zeros((vocab_size, WORD2VEC_EMBED_SIZE))

Loading Word2Vec model and generating embedding matrix...


In [ ]:
embedding_weights

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
for word, index in word2idx.items():
    try:
        embedding_weights[index, :] = word2vec[word.lower()]
    except KeyError:
        pass  # keep as zero (not ideal, but what else can we do?)

In [ ]:
embedding_weights

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.08007812,  0.10498047,  0.04980469, ...,  0.00366211,
         0.04760742, -0.06884766],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

**DEFINING AND TRAINING MODEL**

In [ ]:
import tensorflow as tf
# Get a handle to the attached TPU. On GCP it will be the CloudTPU itself
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# Connect to the TPU handle and initialise it
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.123.44.18:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.123.44.18:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
with strategy.scope():

  qenc = keras.Sequential()
  qenc.add(keras.layers.Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,
                    weights=[embedding_weights], mask_zero=True))
  qenc.add(layers.Bidirectional(layers.LSTM(QA_EMBED_SIZE, input_length=seq_maxlen, return_sequences=False)))
  qenc.add(layers.Dropout(0.3))

In [ ]:
with strategy.scope():

  aenc = keras.Sequential()
  aenc.add(layers.Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,
                    weights=[embedding_weights], mask_zero=True))
  aenc.add(layers.Bidirectional(layers.LSTM(QA_EMBED_SIZE, input_length=seq_maxlen, return_sequences=False)))
  aenc.add(layers.Dropout(0.3))

In [ ]:
from keras.layers import concatenate

with strategy.scope():

  merged = concatenate([qenc.output, aenc.output])
  out = layers.Dense(2, activation="softmax")(merged)
  model = keras.Model(inputs=[qenc.input, aenc.input], outputs=[out])

  model.compile(optimizer="adam", loss="categorical_crossentropy",
                metrics=["accuracy"])
  
'''merged = concatenate([qenc.output, aenc.output])
output = layers.Dense(2, activation="softmax")'''

'merged = concatenate([qenc.output, aenc.output])\noutput = layers.Dense(2, activation="softmax")'

In [ ]:
# Dealing with class imbalance
pos = 0
for i in range(Ytrain.shape[0]):
  # print(Ytrain[i][0])
  if Ytrain[i][0]:
    pos += 1
print(pos)

9265


In [ ]:
# Dealing with class imbalance
total = Xqtrain.shape[0]
#pos = np.count_nonzero(Ytrain)
neg = total - pos
weight_for_0 = (1 / neg) * (total) / 2.0 
weight_for_1 = (1 / pos) * (total) / 2.0
class_weight = {0: weight_for_1, 1: weight_for_0}
print(class_weight)

{0: 4.693200215866163, 1: 0.5596203346203346}


In [ ]:
# class_weights_dict

{0: 4.693200215866163, 1: 0.5596203346203347}

In [ ]:
print("Training...")
checkpoint = ModelCheckpoint(
    filepath=os.path.join(project_folder, "qa-lstm/qa-blstm-best{epoch:08d}.hdf5"),
    verbose=1, save_best_only=True)

'''model.fit([Xqtrain, Xatrain], Ytrain, batch_size=BATCH_SIZE,
          epochs=NBR_EPOCHS, validation_split=0.1, 
          callbacks=[checkpoint])'''

model.fit([Xqtrain, Xatrain], Ytrain, batch_size=BATCH_SIZE,
          epochs=NBR_EPOCHS, validation_data=([Xqtest, Xatest], Ytest), 
          callbacks=[checkpoint], class_weight=class_weight)

Training...
Epoch 1/20
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


   2/2718 [..............................] - ETA: 1:30:57 - loss: 0.5378 - accuracy: 0.3594WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0019s vs `on_train_batch_end` time: 0.4187s). Check your callbacks.


2718/2718 [==============================] - ETA: 0s - loss: 0.6872 - accuracy: 0.5295WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0017s vs `on_test_batch_end` time: 0.1027s). Check your callbacks.



Epoch 00001: val_loss improved from inf to 0.78506, saving model to ./drive/My Drive/csc2515-project/qa-lstm/qa-blstm-best00000001.hdf5
2718/2718 [==============================] - 1237s 455ms/step - loss: 0.6872 - accuracy: 0.5295 - val_loss: 0.7851 - val_accuracy: 0.2747
Epoch 2/20
2718/2718 [==============================] - ETA: 0s - loss: 0.6140 - accuracy: 0.6504
Epoch 00002: val_loss improved from 0.78506 to 0.60832, saving model to ./drive/My Drive/csc2515-project/qa-lstm/qa-blstm-best00000002.hdf5
2718/2718 [==============================] - 1221s 449ms/step - loss: 0.6140 - accuracy: 0.6504 - val_loss: 0.6083 - val_accuracy: 0.6609
Epoch 3/20
2718/2718 [==============================] - ETA: 0s - loss: 0.4088 - accuracy: 0.8039
Epoch 00003: val_loss did not improve from 0.60832
2718/2718 [==============================] - 1208s 445ms/step - loss: 0.4088 - accuracy: 0.8039 - val_loss: 0.7313 - val_accuracy: 0.6127
Epoch 4/20
2718/2718 [==============================] - ETA: 0

In [ ]:
print("Evaluation...")
loss, acc = model.evaluate([Xqtest, Xatest], Ytest, batch_size=BATCH_SIZE)
print("Test loss/accuracy final model = %.4f, %.4f" % (loss, acc))

Evaluation...
  2/272 [..............................] - ETA: 35s - loss: 1.0947 - accuracy: 0.6875WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0015s vs `on_test_batch_end` time: 0.1031s). Check your callbacks.


272/272 [==============================] - 30s 109ms/step - loss: 1.1451 - accuracy: 0.7954
Test loss/accuracy final model = 1.1451, 0.7954


In [ ]:
predicted

array([[4.0797847e-03, 9.9592018e-01],
       [1.3447542e-01, 8.6552459e-01],
       [2.9702608e-03, 9.9702972e-01],
       ...,
       [1.7144256e-04, 9.9982864e-01],
       [5.9002155e-01, 4.0997842e-01],
       [1.1988326e-05, 9.9998796e-01]], dtype=float32)

**SAVE OR LOAD WEIGHTS**

In [ ]:
#model.save_weights(os.path.join(project_folder, "qa-lstm/qa-blstm-final.hdf5"))
#model.load_weights(filepath=os.path.join(project_folder, "qa-lstm/qa-blstm-final.hdf5"))
#loss, acc = model.evaluate([Xqtest, Xatest], Ytest, batch_size=BATCH_SIZE)
#rint("Test loss/accuracy best model = %.4f, %.4f" % (loss, acc))

**PREDICT**

In [ ]:
predicted = model.predict([Xqtest, Xatest])

In [ ]:
best_answers = 0
threshold = 0.8
predictions = []

# [1, 0] - best answer
# [0, 1] - not best answer

for predict in predicted:
  print(predict[0], predict[1])
  if predict[0] > threshold:
    best_answers += 1
    predictions.append(1)
  else:
    predictions.append(0)

Streaming output truncated to the last 5000 lines.
2.016469e-06 0.999998
0.2077951 0.7922049
0.0079240445 0.99207586
0.8995299 0.1004701
0.0019577094 0.9980423
0.2147048 0.7852952
0.0006891817 0.99931085
0.06160889 0.93839103
0.0001820311 0.999818
0.2927737 0.70722634
0.15110049 0.84889954
0.049535837 0.95046425
0.010439482 0.9895605
0.0462617 0.9537383
0.97060984 0.029390184
0.002918833 0.9970811
0.0010759744 0.998924
0.0076521463 0.99234784
1.16240855e-07 0.9999999
1.0435439e-06 0.9999989
2.1131384e-06 0.99999785
0.14425816 0.85574186
0.054100595 0.9458994
0.00017369274 0.9998264
0.11989291 0.88010716
0.34266025 0.6573399
0.5770119 0.42298812
0.0037495138 0.99625045
0.47197336 0.52802664
0.06490027 0.9350997
6.268014e-05 0.9999373
0.20281728 0.79718274
0.0019981416 0.9980019
0.124438986 0.875561
0.0054601473 0.9945399
1.4620683e-06 0.99999857
5.5526136e-05 0.99994445
0.07480169 0.9251983
0.266963 0.733037
0.0006411752 0.99935883
2.8475406e-05 0.9999715
8.717774e-05 0.99991286
0.19553

In [ ]:
questions_answers_test['Predicted_Yes'] = predicted[:, 0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
questions_answers_test['Prediction'] = predictions
best_answers_pred = (questions_answers_test.groupby(["QId"])["Prediction"].idxmax()).tolist()
best_answers_test = (questions_answers_test.groupby(["QId"])["Best_Score"].idxmax()).tolist()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# Calculate accuracy
count=0
correct=0
for i in range(len(best_answers_test)):
  if best_answers_pred[i] ==best_answers_test[i]:
    correct+=1
  count+=1
print(correct/count)

0.22336377473363775
